<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/Ejercicios_solucionboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/institutohumai/machine_learning/blob/main/7_XGBoost/Ejercicios/Ejercicios_solucion.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Ejercicios XGBoost, LightGBM, CatBoost y NGBoost

Recuerden activar la opción de acelerador por hardaware "gpu" La cual se encuentra en "Entorno de ejecución" y dentro de "cambiar tipo de entorno de ejecución"



## Diseño de Modelo XGBoost para predicción de salarios
Utilizando la librería XGBoost:
1. Limpie el Dataset eliminando los valores nulos y conviertendo las variables categóricas mediante OneHotEncoding o similar.
2. Divida el conjunto de datos en datos de entrenamiento y prueba.
3. Implemente un modelo XGBoost para predecir si una persona gana más de 50K en función de las otras características y entrénelo con el set de entrenamiento.
4. Evalúe el rendimiento del modelo usando el conjunto de prueba y la métrica ROC_AUC.


In [3]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [4]:
# Leer el dataset
df = pd.read_csv("https://storage.googleapis.com/humai-datasets/aws_s3/ml_avanzado/2_Boosting_de_arboles/adult_train.csv")

# Limpieza de datos y Encoding
df = df.replace('?', np.nan)
df = df.dropna()
ohe = OneHotEncoder()
df_encoded = ohe.fit_transform(df)

# Data Split
X = df_encoded[:, :-1]  # Todas las columnas excepto la última (target)
y = df_encoded[:, -1]   # Última columna (target)

# Convertir las matrices dispersas a arreglos NumPy
X = X.toarray()
y = y.toarray().ravel()

# División de datos en entrenamiento y prueba, excluyendo la columna objetivo
X_train, X_test, y_train, y_test = train_test_split(X[:, :-1], y, test_size=0.2, random_state=42)

In [5]:
# Modelo
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=42, tree_method='gpu_hist')
xg_cl.fit(X_train, y_train)

# Evaluación
preds = xg_cl.predict(X_test)
auc_score_xgboost = roc_auc_score(y_test, preds)
print('El resultado de ROC AUC es:', auc_score_xgboost)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:30:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:30:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:30:49] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.


El resultado de ROC AUC es: 0.7583436750227517


## Empleo de Lightgbm para predicciones salariales
Implemente y entrene un modelo Lightgbm utilizando el dataset adult para predecir el salario de una persona. Evalúe el rendimiento del modelo usando el conjunto de datos de prueba y la métrica ROC_AUC.

In [6]:
import lightgbm as lgb

# Crear el dataset LightGBM
train_data = lgb.Dataset(X_train, label=y_train)

# Definir los parámetros del modelo LightGBM
params = {
    'objective': 'binary',
    'metric': 'auc',
    'seed': 42,
}

# Entrenar el modelo LightGBM en la GPU
lgbm_cl = lgb.train(params, train_data, num_boost_round=1)

# Evaluación
preds = lgbm_cl.predict(X_test)
auc_score_lightgbm = roc_auc_score(y_test, preds)
print('El resultado de ROC AUC es:', auc_score_lightgbm)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 5978, number of negative: 18151
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009899 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 508
[LightGBM] [Info] Number of data points in the train set: 24129, number of used features: 254
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.247752 -> initscore=-1.110640
[LightGBM] [Info] Start training from score -1.110640
El resultado de ROC AUC es: 0.8684468161385512


## Empleo de CatBoost para predicciones salariales
Implemente y entrene un modelo CatBoost utilizando el dataset adult para predecir el salario de una persona. Evalúe el rendimiento del modelo usando el conjunto de datos de prueba y la métrica ROC_AUC.


In [7]:
!pip install catboost

In [8]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Modelo
catb_cl = CatBoostClassifier(verbose=0, n_estimators=100, task_type='GPU')
catb_cl.fit(X_train, y_train)

# Evaluación
preds = catb_cl.predict(X_test)
auc_score_catboost = roc_auc_score(y_test, preds)
print('El resultado de ROC AUC es:', auc_score_catboost)


El resultado de ROC AUC es: 0.7796044467087302


## Empleo de NGBoost para predicciones salariales
Implemente y entrene un modelo NGBoost utilizando el dataset adult para predecir el salario de una persona. Evalúe el rendimiento del modelo usando el conjunto de datos de prueba y la métrica ROC_AUC.

In [9]:
!pip install ngboost

Esta celda ayuda a que los modelos puedan correr en un colab debido a que reduce la dimensionalidad de los datos. No es necesario igualmente en todos los modelos que realicen. Se utiliza principalmente en el ngboost debido a que no tiene la posibilidad de correr el modelo utilizando la gpu que brinda colab. Hay que tener en cuenta que los resultados se veran afectados al reducir la dimensionalidad del dataset

In [ ]:

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Reducción de dimensionalidad con PCA
pca = PCA(n_components=10)  # Número de componentes principales deseados
X_pca = pca.fit_transform(X[:, :-1])

# División de datos en entrenamiento y prueba con datos reducidos por PCA
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Convertir las etiquetas a formato entero
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


In [ ]:
from ngboost import NGBClassifier
from ngboost.distns import Bernoulli

# Modelo
ngb_cl = NGBClassifier(Dist=Bernoulli, n_estimators=100) # Se especifica Bernoulli como la distribución de la clase Dist para problemas de clasificación binaria
ngb_cl.fit(X_train, y_train)

# Evaluación
preds = ngb_cl.predict_proba(X_test)[:, 1]
auc_score_ngboost = roc_auc_score(y_test, preds)
print('El resultado de ROC AUC es:', auc_score_ngboost)


## Comparación de Modelos
Compare el rendimiento de cada modelo y determine cuál se desempeña mejor en función de la métrica ROC_AUC.


In [ ]:
print(f'XGBoost AUC Score: {auc_score_xgboost}')
print(f'LightGBM AUC Score: {auc_score_lightgbm}')
print(f'CatBoost AUC Score: {auc_score_catboost}')
print(f'NGBoost AUC Score: {auc_score_ngboost}')